[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/FranQuant/the_ai_engineer_capstones/blob/main/capstones/week01_gd_optimization/gd_capstone.ipynb
)

# Gradient Descent Optimization — Week 1 Capstone

>This notebook implements the Week-1 capstone for *The AI Engineer* program using two 1-D objectives.

1. **Quadratic baseline**
   $$
   q(x) = \tfrac{1}{2} x^2
   $$
   A convex reference for studying GD stability and step-size behavior.

2. **Cubic loss**
   $$
   f(x) = x^3 - 3x
   $$
   A non-convex function highlighting multiple stationary points and contrasting GD/SGD dynamics.

We analyze:
- Deterministic **GD**
- **SGD** (constant and diminishing steps: $\eta_t = \frac{\eta_0}{1 + k t}$)
- **Step-size sensitivity**, stability regimes, and reproducibility via a fixed global seed.

## 1. Setup & Hyperparameters

### Summary Table

| Component | Setting | Value / Formula |
|----------|---------|------------------|
| **Global seed** | Fixed integer | $\text{SEED} = 123$ |
| **GD step-sizes (quadratic)** | Sweep | $\eta \in \{0.05,\;0.10,\;0.15,\;0.20\}$ (plus $\eta=2.1$ for an unstable example) |
| **GD step-size (cubic)** | Fixed | $\eta_{\text{cubic}} = 0.05$ |
| **SGD (constant)** | Step-size | $\eta = \eta_0 = 0.05$ |
| **SGD (diminishing)** | Schedule | $\eta_t = \dfrac{\eta_0}{1 + k t}$ |
| **SGD decay constant** | $k$ | $k = 0.01$ |
| **Tolerance** | Objective-gap | $\varepsilon = 10^{-4}$ |
| **Max iterations** | Upper bound | $T_{\max} = 5000$ |

In [ ]:
# ============================================
# 0. Setup & Hyperparameters
# ============================================

import numpy as np
import matplotlib.pyplot as plt

# --- Reproducible Global Seed ---
SEED = 123
rng = np.random.default_rng(SEED)

# --- Learning Rates ---
# Quadratic baseline sweep (for stability analysis)
ETA_QUAD_SWEEP = [0.05, 0.10, 0.15, 0.20, 2.1] # 2.1 is outside 0<η<2, so GD diverges

# GD on the cubic objective
ETA_CUBIC = 0.05

# --- SGD Settings ---
ETA0 = 0.05            # constant-step SGD (η)
ETA_SGD = ETA0         # alias for clarity inside SGD functions
K_SCHEDULE = 0.01      # diminishing-step decay factor k
SIGMA_SGD = 0.5        # Gaussian noise scale σ

# --- Global Experiment Settings ---
EPS = 1e-4             # convergence tolerance on objective gap
T_MAX = 5000           # iteration cap for GD/SGD

# --- Plot Styling ---
plt.style.use("seaborn-v0_8-darkgrid")
plt.rcParams["figure.figsize"] = (7, 4)
plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.spines.right"] = False

## 2. Quadratic Baseline: Objective & Derivative

We begin with the convex quadratic

$$
q(x) = \tfrac{1}{2} x^2,
$$

which has a simple derivative

$$
q'(x) = x.
$$

This function is ideal for illustrating:

>- **Stability** vs **instability** under different step sizes  
>- Exact relationship between curvature and step-size limits  
>- How GD behaves in a perfectly smooth, convex landscape

The unique minimizer is

$$
x^\star = 0.
$$

> Before running gradient descent, we implement the objective, derivative, and a helper function to compute the objective gap

$$
\text{gap}(x_t) = q(x_t) - q(x^\star).
$$

In [ ]:
# ============================================
# 1. Quadratic Baseline: Objective & Derivative
# ============================================

def q(x):
    """Quadratic objective: q(x) = 0.5 * x^2"""
    return 0.5 * x**2

def dq(x):
    """Derivative of the quadratic: q'(x) = x"""
    return x

def quad_gap(x):
    """Objective gap relative to optimum x* = 0."""
    return q(x) - q(0.0)

## 3. Gradient Descent on the Quadratic — Step-Size Sweep

>For the quadratic
>$$q(x) = \tfrac{1}{2}x^2$$
>Gradient Descent follows the update rule
>$$x_{t+1} = x_t - \eta\, q'(x_t) = x_t - \eta x_t = (1 - \eta)x_t$$
>
>This yields an exact stability condition:
>$$|1 - \eta| < 1 \quad \Longleftrightarrow \quad 0 < \eta < 2$$

A step-size sweep is ideal to illustrate:

>- **Slow convergence** for small $\eta$
>- **Fast convergence** near the optimal $\eta = 1$
>- **Oscillations** for $1 < \eta < 2$
>- **Divergence** for $\eta \ge 2$
>
>We run GD from the same initialization
>$$x_0 = 4$$
>and compare trajectories for:
>$$\eta \in \{0.05,\; 0.10,\; 0.15,\; 0.20, 2.1\}$$
>
>We include an additional unstable step size:
>
>- Stable: 0.05, 0.10, 0.15, 0.20  
>- Unstable: **2.1** → divergence (violates $0 < \eta < 2$)
>  
>GD on a quadratic with curvature 1 is stable only if:
>$$0 < \eta < 2$$

In [ ]:
# ============================================
# 2. GD on Quadratic — Step-Size Sweep (Two-Pane Figure)
# ============================================

def gd_quadratic(x0, eta, T=T_MAX):
    """Runs GD on the quadratic for T steps (with gap-based stopping + divergence guard)."""
    xs = [x0]
    x = x0

    for _ in range(T):
        x = x - eta * dq(x)

        # --- Divergence guard (critical!) ---
        if abs(x) > 1e6:
            xs.append(np.nan)
            break

        xs.append(x)

        # --- Gap-based stopping ---
        if abs(q(x) - q(0.0)) < EPS:
            break

    return np.array(xs)

# ============================================
# 2. GD on the Quadratic — Stable vs Unstable
# ============================================

x0_quad = 4.0
stable_etas = [0.05, 0.10, 0.15, 0.20]
unstable_eta = 2.1

fig, ax = plt.subplots(2, 1, figsize=(8, 8))

# --- Top panel: stable ---
for eta in stable_etas:
    xs = gd_quadratic(x0_quad, eta)
    ax[0].plot(xs, label=f"η = {eta}")

ax[0].set_title("Quadratic GD — Stable Step Sizes (0 < η < 2)")
ax[0].set_xlabel("Iteration")
ax[0].set_ylabel("x_t")
ax[0].legend()

# --- Bottom panel: divergent ---
xs_div = gd_quadratic(x0_quad, unstable_eta)
ax[1].plot(xs_div, color="purple", label=f"η = {unstable_eta}")

ax[1].set_title("Quadratic GD — Divergence (η ≥ 2)")
ax[1].set_xlabel("Iteration")
ax[1].set_ylabel("x_t")
ax[1].legend()

plt.figtext(
    0.5, -0.02,
    "Stable η values decay smoothly; η = 2.1 lies outside 0 < η < 2 and diverges geometrically.",
    ha="center", fontsize=10,
)

plt.tight_layout()
plt.show()

Thus, $\eta = 0.05, 0.10, 0.15, 0.20$ all converge to $x^\star = 0$ (with faster convergence
as $\eta$ grows), while $\eta = 2.1$ lies outside the stability band and produces true divergence.

### Quadratic GD — Convergence Metrics

We compute simple convergence diagnostics for the quadratic baseline.

- **Final gap**  
  $$|q(x_T) - q(0)|$$

- **Best gap**  
  $$\min_{t \le T} |q(x_t) - q(0)|$$

- **Steps-to-tolerance**  
  $$|q(x_t) - q(0)| < 10^{-4}$$

In [ ]:
# ============================================
# Quadratic Metrics
# ============================================

def compute_quad_metrics(xs, eps=EPS):
    """Compute final gap, best gap, and steps-to-tolerance for q(x)."""
    gaps = np.abs(q(xs) - q(0.0))

    final_gap = gaps[-1]
    best_gap = gaps.min()

    idx = np.where(gaps < eps)[0]
    steps_to_tol = int(idx[0]) if len(idx) > 0 else None

    return final_gap, best_gap, steps_to_tol


# Run GD on the quadratic with a representative step size
xs_quad_metrics = gd_quadratic(4.0, eta=0.05)

quad_metrics = compute_quad_metrics(xs_quad_metrics)

print("=== Convergence Metrics (Quadratic Objective) ===")
print(f"GD (eta=0.05):")
print(f"  Final gap        = {quad_metrics[0]:.6f}")
print(f"  Best gap         = {quad_metrics[1]:.6f}")
print(f"  Steps-to-tol     = {quad_metrics[2]}")

## 4. Cubic Objective: Non-Convex Landscape & Derivative

We now introduce the non-convex cubic function

> $$f(x) = x^3 - 3x,\quad f'(x) = 3x^2 - 3.$$


### Stationary points

>Set $f'(x) = 0$:
>
>$$3x^2 - 3 = 0 \quad \Longrightarrow \quad x = \pm 1$$
>
>- **$x = -1$** is a **local maximum** (unstable under GD).  
>- **$x = +1$** is a **local minimum** (stable for $0 < \eta < \tfrac{1}{3}$).

### Basin structure

>For positive step sizes:
>
>- Ix  $x_0 > -1$  GD flows to the stable minimizer $x=1$
>- If $x_0 < -1$, the gradient pushes the iterate toward $-\infty$ unless a guard prevents it.
>
>- The point $x=-1$ is a **unstable equilibrium of measure zero**.

**This makes the cubic ideal for illustrating:**

>- **Stability vs. instability** of critical points  
>- **Basymmetric basins of attraction**  
>- **Sensitivity to initializatione**  
>- Differences between GD and noisy **SGD**
  
Below we implement the function, gradient, and the objective-gap around the minimizer at $x^\star = 1$

In [ ]:
# ============================================
# 3. Cubic Objective: f(x) = x^3 - 3x
# ============================================

def f(x):
    """Cubic objective: f(x) = x^3 - 3x"""
    return x**3 - 3*x

def df(x):
    """Derivative: f'(x) = 3x^2 - 3"""
    return 3*x**2 - 3

def cubic_gap(x):
    """Objective gap relative to the stable minimizer x* = +1."""
    return f(x) - f(1.0)

### Cubic Objective — Function and Derivative
*We briefly inspect the cubic landscape to capture its non-convex geometry before running optimization.*

In [ ]:
# ============================================
# Cubic Diagnostics: f(x) and f'(x)
# ============================================

xs = np.linspace(-3, 3, 400)

fig, ax = plt.subplots(1, 2, figsize=(12, 4))

# --- f(x) plot ---
ax[0].plot(xs, f(xs), linewidth=2)
ax[0].axvline(-1, linestyle="--", color="grey", label="x = -1 (max)")
ax[0].axvline(1, linestyle="--", color="black", label="x = 1 (min)")
ax[0].set_title("Cubic Objective  $f(x) = x^3 - 3x$")
ax[0].set_xlabel("x")
ax[0].set_ylabel("f(x)")
ax[0].legend()

# --- f'(x) plot ---
ax[1].plot(xs, df(xs), linewidth=2, color="darkorange")
ax[1].axhline(0, color="black", linewidth=1)
ax[1].axvline(-1, linestyle="--", color="grey")
ax[1].axvline(1, linestyle="--", color="black")
ax[1].set_title("Derivative  $f'(x) = 3x^2 - 3$")
ax[1].set_xlabel("x")
ax[1].set_ylabel("f'(x)")

plt.figtext(
    0.5, -0.05,
    "The cubic has a saddle-like region around x=-1 and strong curvature near x=±∞.\n"
    "This explains sensitivity of GD to initialization and the steady-state variance of SGD.",
    ha="center", fontsize=9
)

plt.show()

## 4. Gradient Descent on the Cubic — Trajectories & Basins

For the cubic

$$
f(x) = x^3 - 3x,
\quad
f'(x) = 3x^2 - 3,
$$
>
>Gradient Descent follows the update rule
>
>$$x_{t+1} = x_t - \eta f'(x_t)$$
>
>There are two stationary points:
>
>- **$x = -1$** — local maximum (repelling under GD)  
>- **$x = +1$** — local minimum (attracting when $0 < \eta < 1/3$)

### Basin structure

The cubic has a **single attracting basin**:

>- If $x_0 > -1$
>
>- GD converges to $ x^\star = 1 $.
>
>- If $x_0 < -1,$
>
>  GD diverges to $ -\infty $ unless a guard prevents it.

We run GD from several initial points to visualize this behavior:

>$$x_0 \in \{-3,\; -0.5,\; 0.5,\; 2,\; 1.5\}$$
>
>We use the fixed step-size:
>
>$$\eta_{\text{cubic}} = 0.05.$$

The resulting trajectories reveal the basin boundary at $x = -1$, the stable attractor at $x = 1$, and divergence when starting too far left.

In [ ]:
# ============================================
# 4. GD on the Cubic — Multiple Initializations
# ============================================

def gd_cubic(x0, eta=ETA_CUBIC, T=T_MAX):
    """Runs GD on the cubic for T steps (with guard for divergence and gap-based early stop)."""
    xs = [x0]
    x = x0

    for _ in range(T):
        x = x - eta * df(x)

        # ---- GAP-BASED STOPPING (relative to minimum at x*=1) ----
        if abs(f(x) - f(1.0)) < EPS:      # This is the spec requirement
            xs.append(x)
            break

        # ---- DIVERGENCE GUARD ----
        if abs(x) > 20:
            xs.append(np.nan)
            break

        xs.append(x)

    return np.array(xs)


# Initializations to test
inits = [-3.0, -0.5, 0.5, 2.0, 1.5]

plt.figure(figsize=(7, 4))
for x0 in inits:
    xs = gd_cubic(x0, ETA_CUBIC)
    plt.plot(xs[:20], label=f"x0 = {x0}")

plt.axhline(1.0, color="black", linewidth=1, linestyle="--", label="x* = 1")
plt.title("GD on the Cubic — Basin Behavior")
plt.xlabel("Iteration")
plt.ylabel("x_t")
plt.legend()

plt.figtext(
    0.5, -0.1,
    "GD trajectories on the cubic. Initializations right of -1 converge to x*=1; "
    "starting left of -1 diverges without guards.",
    ha="center", fontsize=9
)

plt.show()

## 5. Stochastic Gradient Descent (SGD) on the Cubic

SGD replaces the exact gradient  
$$f'(x) = 3x^2 - 3$$  
with a noisy estimate:
$$
g_t = f'(x_t) + \varepsilon_t, 
\qquad \varepsilon_t \sim \mathcal{N}(0,\sigma^2).
$$

This induces **wandering** around the minimizer $x^\star = 1$, with steady-state variance
$$
\mathrm{Var}(x_t) \propto \eta\,\sigma^2,
\qquad \text{radius} \sim \sqrt{\eta}\,\sigma.
$$

We use two SGD schedules:

1. **Constant step-size**  
   $$x_{t+1} = x_t - \eta\, g_t$$  
   Converges only to a *neighborhood* of $x^\star$.

2. **Diminishing step-size**  
   $$\eta_t = \frac{\eta_0}{1 + k t}$$  
   $$x_{t+1} = x_t - \eta_t\, g_t$$  
   Gradually reduces noise and tightens convergence around the minimizer.

Both variants rely on a shared global RNG for reproducibility.


In [ ]:
# ============================================
# 5. SGD on the Cubic — Constant & Diminishing Step (Shared RNG Version)
# ============================================

def sgd_constant(x0, eta=ETA_SGD, sigma=SIGMA_SGD, T=T_MAX, rng=rng):
    """SGD with constant step size + gap-based stopping."""
    xs = [x0]
    x = x0

    for _ in range(T):
        noise = rng.normal(0, sigma)
        x = x - eta * (df(x) + noise)

        # ---- GAP-BASED STOPPING ----
        if abs(f(x) - f(1.0)) < EPS:
            xs.append(x)
            break

        # ---- DIVERGENCE GUARD ----
        if abs(x) > 20:
            xs.append(np.nan)
            break

        xs.append(x)

    return np.array(xs)



def sgd_diminishing(x0, eta0=ETA0, k=K_SCHEDULE, sigma=SIGMA_SGD, T=T_MAX, rng=rng):
    """SGD with diminishing steps + gap-based stopping."""
    xs = [x0]
    x = x0

    for t in range(T):
        eta_t = eta0 / (1.0 + k * t)
        noise = rng.normal(0, sigma)
        x = x - eta_t * (df(x) + noise)

        # ---- GAP-BASED STOPPING ----
        if abs(f(x) - f(1.0)) < EPS:
            xs.append(x)
            break

        # ---- DIVERGENCE GUARD ----
        if abs(x) > 20:
            xs.append(np.nan)
            break

        xs.append(x)

    return np.array(xs)

## 6. SGD Trajectories on the Cubic

We now compare **constant-step** and **diminishing-step** SGD on the cubic objective  
starting from the same initialization $x_0 = 0.5$.

- With a **constant step**, SGD converges only to a *noisy neighborhood* around  
  $x^\star = 1$, with fluctuations scaling like  
  $$\text{radius} \propto \eta^{1/2}\,\sigma.$$

- With a **diminishing schedule**  
  $$\eta_t = \frac{\eta_0}{1 + k t},$$  
  the noise gradually shrinks, yielding a tighter approach to the minimizer.

The plot below compares both trajectories over $T = 200$ iterations.


In [ ]:
# ============================================
# 6. SGD Trajectories — Constant vs Diminishing
# ============================================

T_plot = 200
x0 = 0.5

xs_const = sgd_constant(x0, eta=ETA_SGD, sigma=SIGMA_SGD, T=T_plot)
xs_dimin = sgd_diminishing(x0, eta0=ETA0, k=K_SCHEDULE, sigma=SIGMA_SGD, T=T_plot)

plt.figure(figsize=(8, 4))
plt.plot(xs_const, label="Constant SGD", linewidth=2)
plt.plot(xs_dimin, label="Diminishing-step SGD", linewidth=2)
plt.axhline(1.0, color="black", linestyle="--", linewidth=1, label="x* = 1")

plt.title("SGD on the Cubic — Constant vs Diminishing Step")
plt.xlabel("Iteration")
plt.ylabel("x_t")
plt.legend()

plt.figtext(
    0.5, -0.12,
    "Constant-step SGD fluctuates around x*=1; diminishing-step SGD converges tighter as eta_t decreases.",
    ha="center", fontsize=9
)

plt.show()

## 7. Metrics: Final Gap, Best Gap, and Steps-to-Tolerance

For each optimization method, we report three quantities relative to the cubic
objective’s stable minimizer at $x^\star = 1$:

- **Final gap:**  
  $$| f(x_T) - f(x^\star) |$$

- **Best gap:**  
  $$\min_{t \le T} \; |f(x_t) - f(x^\star)| $$

- **Steps-to-tolerance:**  
  The first iteration \(t\) such that  
  $$|f(x_t) - f(x^\star)| < \varepsilon, \qquad \varepsilon = 10^{-4}$$

These metrics quantify the speed, stability, and noise behavior of each optimizer.

In [ ]:
# ============================================
# 7. Metrics: Gaps and Steps-to-Tolerance
# ============================================

def compute_metrics(xs, objective, x_star=1.0, eps=EPS):
    """
    Compute final gap, best gap, and steps-to-tolerance.
    Includes divergence detection via NaN check.
    """
    xs = np.asarray(xs)

    # Divergence / instability detection
    if np.isnan(xs).any():
        return np.nan, np.nan, None

    gaps = np.abs(objective(xs) - objective(x_star))

    final_gap = gaps[-1]
    best_gap = gaps.min()

    idx = np.where(gaps < eps)[0]
    steps_to_tol = int(idx[0]) if len(idx) > 0 else None

    return final_gap, best_gap, steps_to_tol

### Results

In [ ]:
# ----- Run trajectories -----
# GD
xs_gd = gd_cubic(0.5, ETA_CUBIC)

# SGD constant
xs_sgd_const = sgd_constant(0.5, eta=ETA_SGD, sigma=SIGMA_SGD)

# SGD diminishing
xs_sgd_dim = sgd_diminishing(0.5, eta0=ETA0, k=K_SCHEDULE, sigma=SIGMA_SGD)

# ----- Compute metrics -----

gd_metrics     = compute_metrics(xs_gd, f)
sgd_c_metrics  = compute_metrics(xs_sgd_const, f)
sgd_d_metrics  = compute_metrics(xs_sgd_dim, f)

# ----- Display -----
print("=== Convergence Metrics (Cubic Objective) ===")
print(f"GD (eta={ETA_CUBIC}):")
print(f"  Final gap        = {gd_metrics[0]:.6f}")
print(f"  Best gap         = {gd_metrics[1]:.6f}")
print(f"  Steps-to-tol     = {gd_metrics[2]}")

print("\nSGD Constant Step:")
print(f"  Final gap        = {sgd_c_metrics[0]:.6f}")
print(f"  Best gap         = {sgd_c_metrics[1]:.6f}")
print(f"  Steps-to-tol     = {sgd_c_metrics[2]}")

print("\nSGD Diminishing Step:")
print(f"  Final gap        = {sgd_d_metrics[0]:.6f}")
print(f"  Best gap         = {sgd_d_metrics[1]:.6f}")
print(f"  Steps-to-tol     = {sgd_d_metrics[2]}")

## 8. Final Commentary — Key Takeaways

### Gradient Descent (GD)

- Deterministic update rule:
  $$
  x_{t+1} = x_t - \eta f'(x_t)
  $$
- Converges cleanly and rapidly to the stable minimizer \(x^\star = 1\).
- Serves as a model of *idealized batch optimization* with no noise.

### Stochastic Gradient Descent — Constant Step

- Uses noisy gradients:
  $$
  g_t = f'(x_t) + \varepsilon_t, 
  \qquad \varepsilon_t \sim \mathcal{N}(0, \sigma^2)
  $$
- Does **not** converge to the exact minimizer, but to a *noise ball* around it.
- Steady-state radius scales as:
  $$
  \text{radius} \;\propto\; \sqrt{\eta}\,\sigma.
  $$
- Mimics **stochastic training behavior** seen in deep learning.

### Stochastic Gradient Descent — Diminishing Step

- Decaying schedule:
  $$
  \eta_t = \frac{\eta_0}{1 + k t}
  $$
- Shrinks noise gradually, tightening convergence around $x^\star$.
- Behaves like **annealing**, **fine-tuning**, or **long-run stabilizing schedules** used in modern training pipelines.